# Imports

In [ ]:
import os
import cv2
import shutil
import numpy as np
import dlib

# Split fake images into easy, mid, hard 

In [ ]:
source_folder = './Data_files/full_ds/fake'
destination_folder = './Data_files/Augmented_ds'

# Create the destination subfolders
os.makedirs(os.path.join(destination_folder, 'easy'), exist_ok=True)
os.makedirs(os.path.join(destination_folder, 'mid'), exist_ok=True)
os.makedirs(os.path.join(destination_folder, 'hard'), exist_ok=True)

for filename in os.listdir(source_folder):
    if filename.startswith('easy'):
        shutil.move(os.path.join(source_folder, filename), os.path.join(destination_folder, 'easy', filename))
    elif filename.startswith('mid'):
        shutil.move(os.path.join(source_folder, filename), os.path.join(destination_folder, 'mid', filename))
    elif filename.startswith('hard'):
        shutil.move(os.path.join(source_folder, filename), os.path.join(destination_folder, 'hard', filename))

print("Images organized successfully!")

# Apply Augmentation and Crop face on fake images

In [ ]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("./shape_predictor_68_face_landmarks.dat") 

# Define augmentations
def apply_faceshifter(image):
    return cv2.flip(image, 1) 

def width_shift(image, shift_pixels=10):
    rows, cols, _ = image.shape
    M = np.float32([[1, 0, shift_pixels], [0, 1, 0]])
    return cv2.warpAffine(image, M, (cols, rows))

def height_shift(image, shift_pixels=10):
    rows, cols, _ = image.shape
    M = np.float32([[1, 0, 0], [0, 1, shift_pixels]])
    return cv2.warpAffine(image, M, (cols, rows))

def rotate_image(image, angle=15):
    rows, cols, _ = image.shape
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    return cv2.warpAffine(image, M, (cols, rows))

def crop_face(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  
    faces = detector(gray)

    if len(faces) == 0:
        return image  

    for face in faces:
        x = face.left()
        y = face.top()
        w = face.width()
        h = face.height()

        # Ensure the dimensions are within bounds
        if x < 0 or y < 0 or x + w > image.shape[1] or y + h > image.shape[0]:
            return image  

        return image[y:y + h, x:x + w]
    
folder_path = "./Data_files/Augmented_ds/easy"
# folder_path = "./Data_files/Augmented_ds/mid"
# folder_path = "./Data_files/Augmented_ds/hard"

for filename in os.listdir(folder_path):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(folder_path, filename)

        image = cv2.imread(image_path)

        cropped_original = crop_face(image)
        cropped_original_path = os.path.join(folder_path, f"{os.path.splitext(filename)[0]}_cropped.jpg")
        cv2.imwrite(cropped_original_path, cropped_original)
        os.remove(image_path)

        # Perform augmentations
        augmentations = {
            "faceshifter": apply_faceshifter(image.copy()),
            "width_shift": width_shift(image.copy(), shift_pixels=10),
            "height_shift": height_shift(image.copy(), shift_pixels=10),
            "rotate": rotate_image(image.copy(), angle=15)
        }

        # Crop faces and save augmented images
        for aug_name, aug_image in augmentations.items():
            cropped_image = crop_face(aug_image)
            aug_image_path = os.path.join(folder_path, f"{os.path.splitext(filename)[0]}_{aug_name}.jpg")
            cv2.imwrite(aug_image_path, cropped_image)

print("Augmentation completed!")

for filename in os.listdir(folder_path):
    if filename.endswith('.jpg') and not any(descriptor in filename for descriptor in ['width_shift', 'cropped', 'rotate', 'height_shift', 'faceshifter']):
        file_path = os.path.join(folder_path, filename)
        os.remove(file_path)
        print(f"Removed: {file_path}")

# Apply only face crop on real images

In [ ]:
def crop_faces_from_folder(folder_path):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("./shape_predictor_68_face_landmarks.dat")  

    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.jpeg', '.png')): 
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)

            if image is None:
                print(f"Error loading image: {filename}")
                continue

            faces = detector(image, 1)

            if len(faces) == 0:
                print(f"No faces detected in: {filename}")
                os.remove(image_path)  
                continue

            for i, face in enumerate(faces):
                x = face.left()
                y = face.top()
                w = face.right() - x
                h = face.bottom() - y

                if x < 0 or y < 0 or w <= 0 or h <= 0:
                    print(f"Invalid face coordinates for: {filename}")
                    continue

                cropped_face = image[y:y+h, x:x+w]

                if cropped_face.size > 0:
                    cropped_face_path = os.path.join(folder_path, f"{filename}_face_{i+1}.jpg")
                    cv2.imwrite(cropped_face_path, cropped_face)

                else:
                    print(f"Cropped face is empty for: {filename}")

            # Delete the original image
            os.remove(image_path)
            print(f"Processed and removed: {filename}")

folder_path = './Data_files/full_ds/real' 
crop_faces_from_folder(folder_path)